# Prep

## Import Packages and Mount Drive 

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob, json, zipfile, os

import pandas as pd
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import glob
import json
import pandas as pd
import re
import nltk
import copy as cp
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet') 
nltk.download('vader_lexicon')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm.notebook import tqdm as tq
tq.pandas()
import gensim.corpora as corpora
from gensim.models import CoherenceModel, LdaMulticore
from gensim.utils import simple_preprocess 
import gensim.corpora as corpora
from google.colab import files
analyzer = SentimentIntensityAnalyzer()




/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Initalize Paths

In [ ]:
CHECKPOINT_PATH = '/content/drive/MyDrive/UCLA REU 2021 KG /Topic Modeling/NMF/months/'

## Initialize Checkpoint I/O

In [ ]:
def write_data(filename, df, path):
  
  with open(path + '/'+ filename, 'a') as f: 
      result = df.to_json(orient = 'records', double_precision = 15)
      parsed = json.loads(result)
      f.write(json.dumps(parsed))
  f.close()

def load_checkpoint(filename, path):
  
  with open(path + '/'+ filename, 'r') as f: 
    read = pd.read_json(f, 
                        orient='records', keep_default_dates=False, precise_float=True) 
  return pd.DataFrame(read)





## Load Data

In [ ]:
# cleaned_df = load_checkpoint('document_data_mar16-apr07.json','/content/drive/MyDrive/UCLA REU 2021 KG /results')
cleaned_df = load_checkpoint('processed_tweets_20mar11-21june16.json','/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16')

In [ ]:
with open('/content/drive/MyDrive/UCLA REU 2021 KG /results/2020Mar11-2021June16/nmf_tweet_data_kw_senti_20mar11-21jun16_(100,20).json','r') as file:
  data = json.load(file)

In [ ]:
with open(CHECKPOINT_PATH+'/W_(100,20).json','r') as file:
  W = [json.loads(line) for line in file]

In [ ]:
print(W[0])

[0.004532880321274728, 0.00020720658069324152, 0.0, 0.0, 0.0, 0.0, 0.0, 0.31634372711467745, 0.0, 0.0, 0.022516416127119333, 0.0, 0.0, 0.0, 0.008660488441208578, 0.010964927627729248, 0.0, 0.0, 0.004973057091413448, 0.0, 0.0039022181979092726, 0.49201421858406574, 0.0004348708772802355, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00941448141869875, 0.0, 0.0, 0.0010122221626522158, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05293271412643608, 0.0, 0.0, 0.0007192764658308471, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005277580591659191, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0059205486155976365, 0.0, 0.0, 0.0, 0.0, 0.05453490097674192, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0008282694455216886, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00480999523349028, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
topic_threshold = 0.01
topic_minimum = 2 
def cleanTopics(topics):
  nTopics = normalizeWeight(list(topics))
  return [[i,nTopics[i]] for i in range(len(nTopics)) 
    if ((i<topic_minimum) or (nTopics[i]>=topic_threshold))]

processedW = [cleanTopics(w) for w in W]

In [ ]:
for i in range(len(data)):
  data[i]['nmf_topics'] = [v[0] for v in processedW[i]]
  data[i]['nmf_topic_distribution'] = processedW[i]

In [ ]:
print(data[0]['nmf_topic_distribution'])

[[0, 0.004532880321274729], [1, 0.00020720658069324155], [7, 0.3163437271146775], [10, 0.022516416127119337], [15, 0.01096492762772925], [21, 0.4920142185840658], [53, 0.05293271412643609], [76, 0.05453490097674193]]


In [ ]:
with open(CHECKPOINT_PATH+'/NMF_data.json','w') as file:
  json.dump(data, file)

In [ ]:
def vaderScore(text):
  return analyzer.polarity_scores(text)['compound']

In [ ]:
for tweet in data:
  tweet['sentiment_score'] = vaderScore(tweet['text'])

0.0


In [ ]:
print(data[5]['sentiment_score'])

0.5267


In [ ]:
import calendar
print(data[0]['timestamp'])
print(data[0]['timestamp'][4:8])
print(data[0]['timestamp'][-4:])
print(data[0]['timestamp'][4:8]+data[0]['timestamp'][-4:])

Wed Mar 11 07:01:18 +0000 2020
Mar 
2020
Mar 2020
{'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
Oct 2020
202106


In [ ]:
month_dict = {month: index for index, month in enumerate(calendar.month_abbr) if month}
month_list = list(month_dict)

# timestamp to text: 202003->Mar 2020
def timeToText(time:int):
  year = time//100
  month = time%100
  return month_list[month-1]+' '+str(year)

def textToTime(text:str):
  year = text[-4:]
  month = text[:-5]
  return 100*int(year)+month_dict[month]

for tweet in data:
  

## Keyword Extraction using TFIDF


Extracts top $n$ keywords for each tweet. 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from operator import itemgetter

docs = cp.copy(cleaned_df)['cleaned_text'].to_list()
all_text = []
for tweet in docs:
  for word in tweet:
    all_text.append(word)

id2word = corpora.Dictionary(docs)
# Ceate Corpus
texts = docs

corpus = [id2word.doc2bow(text) for text in texts]

vectorizer = TfidfVectorizer()

for i in range(len(texts)):
  texts[i] = ' '.join(texts[i])

tfidf = vectorizer.fit_transform(texts)

feature_names = vectorizer.get_feature_names()

tc = tfidf.tocoo()

doc_dict = {}
for w,d,s in zip(tc.col, tc.row, tc.data):
  word_id = w
  word = feature_names[word_id]
  doc_num = d
  score = s 
  if doc_num in doc_dict.keys():
    doc_dict[doc_num][word] = score
  else:
    doc_dict[doc_num] = {word: score} 

tweets_df = cp.copy(cleaned_df)
tweets_df['keywords'] = tweets_df.apply(lambda x: [], axis=1)


N = 3
all_keywords = []
for doc in doc_dict.keys():
  res = dict(sorted(doc_dict[doc].items(), key = itemgetter(1), reverse = True)[:N])

  keywords = list(res.keys())

  all_keywords.append(keywords)
  tweets_df.at[doc, 'keywords']= keywords


In [ ]:
# write_data('tweets_keywords.json', tweets_df, CHECKPOINT_PATH)
# print("Successfully Saved Checkpoint #3")

Successfully Saved Checkpoint #3


## Bag of Words

Creates corpus in bag-of-words format

In [ ]:
# Create set of words from all tweets
docs = tweets_df['cleaned_text'].to_list()
all_text = []
for tweet in docs:
  for word in tweet:
    all_text.append(word)

# Create Dictionary 
id2word = corpora.Dictionary(docs)

# Ceate Corpus
texts = docs

# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]


## Helper Functions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Use sklearn model
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(n_components//5, 5, figsize=(30, int(1.5*n_components)), sharex=True) # change num of topics?
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f'Topic {topic_idx}',
                     fontdict={'fontsize': 30})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=20)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

# Use lists of dict, each dict contains keyword-weight pairs
def plot_top_words_dict(topics, title):
  fig, axes = plt.subplots(n_components//5, 5, figsize=(30, int(1.5*n_components)), sharex=True) # change num of topics?
  axes = axes.flatten()
  indx = 0 # topic index
  for topic in topics:
    keywords = list(topic)
    weights = [topic[word] for word in keywords]

    ax = axes[indx]
    ax.barh(keywords, weights, height=0.7)
    ax.set_title(f'Topic {indx}',
                  fontdict={'fontsize': 30})
    ax.invert_yaxis()
    ax.tick_params(axis='both', which='major', labelsize=20)
    for i in 'top right left'.split():
        ax.spines[i].set_visible(False)
    fig.suptitle(title, fontsize=40)
    indx+=1


def normalizeWeight(arr): # so that they sum to 1
  return [x/sum(arr) for x in arr] if sum(arr) else []

# NMF



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF

# Change these parameters as necessary
# n_samples = 2000
# n_features = 1000
n_components = 100
n_top_words = 20


In [ ]:
# Frobenius norm
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5) #.fit(tfidf)

W = nmf.fit_transform(tfidf)
# Time complexity:
# Sublinear in timeframe (maybe linear to amount of text?)
#
# (n_components,n_top_words) = (20, 20):
#   101s for ~66 weeks; [12s for 3 weeks;] 
# (30,20): 
#   642s
# (100,20):
#   3509s
# around O(n_compnents^1.5~1.7)


In [ ]:
topic_threshold = 0.01
topic_minimum = 2 
def cleanTopics(topics):
  nTopics = normalizeWeight(list(topics))
  return [[i,nTopics[i]] for i in range(len(nTopics)) 
    if ((i<topic_minimum) or (nTopics[i]>=topic_threshold))]

processedW = [cleanTopics(w) for w in W]

In [ ]:
tweets_df['nmf_topics'] = [[v[0] for v in w] for w in processedW]
tweets_df['nmf_topic_distribution'] = processedW

In [ ]:
print(len(W))
print(len(tweets_df))
for tweet in tweets_df:
  print(tweet)

613949
613949
tweet-id
user-id
user-screen_name
verified
text
cleaned_text
hashtags
mentions
in-reply-to
quoted
timestamp
followers-count
replies_count
quote_count
replies
quotes
keywords


In [ ]:
with open(CHECKPOINT_PATH+'/W_('+str(n_components)+','+str(n_top_words)+').json', 'w') as f:
  for w in W:
    f.write(str(normalizeWeight(w)))
    f.write('\n')

In [ ]:
def normalizeWeight(arr): # so that they sum to 1
  return [x/sum(arr) for x in arr] if sum(arr) else []

W' = [normalizeWeight(w) for w in W]

with open(CHECKPOINT_PATH+'W\'_('+str(n_components)+','+str(n_top_words)+').json', 'w') as f:
  f.write('\n'.join([str(w) for w in W']))

In [ ]:
# generalized Kullback-Leibler divergence
# nmf = NMF(n_components=n_components, random_state=1,
#           beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
#           l1_ratio=.5).fit(tfidf) 

# 15s

# FN looks better for now

In [ ]:
tfidf_feature_names = vectorizer.get_feature_names()
plot_top_words(nmf, tfidf_feature_names, n_top_words,
               'Topics in NMF model (Frobenius norm)')

NameError: ignored

## Store in DataFrame


In [ ]:
def normalizeWeight(arr): # so that they sum to 1
  return [x/sum(arr) for x in arr] if sum(arr) else []

results = []
i=0
for topic_idx, topic in enumerate(nmf.components_):
  top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
  top_features = [feature_names[j] for j in top_features_ind]
  weights = normalizeWeight(topic[top_features_ind])
  results.append({'topic_id':i,'keywords':top_features,'weightings':weights,
                  'keywords_weightings':[(top_features[j],weights[j]) for j in range(len(top_features))],
                  'keywords_dict':{top_features[j]:weights[j] for j in range(len(top_features))}})
  # format of keywords_weightings?
  i+=1

topic_df = pd.DataFrame(results)

In [ ]:
plot_top_words_dict([dct['keywords_dict'] for dct in results][:10],'NMF topics')

# Export Data


In [ ]:
write_data('nmf_topic_data_20mar11-21jun16_('+str(n_components)+','+str(n_top_words)+').json', topic_df, CHECKPOINT_PATH)

In [ ]:
write_data('nmf_tweet_data_20mar11-21jun16_('+str(n_components)+','+str(n_top_words)+').json', cleaned_df, CHECKPOINT_PATH)

# Combined block

In [ ]:
n_components = 400

# Calculation
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)

tfidf_feature_names = vectorizer.get_feature_names()

# Convert result
results = []
i=0
for topic_idx, topic in enumerate(nmf.components_):
  top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
  top_features = [feature_names[j] for j in top_features_ind]
  weights = normalizeWeight(topic[top_features_ind])
  results.append({'topic_id':i,'keywords':top_features,'weightings':weights,
                  'keywords_weightings':[(top_features[j],weights[j]) for j in range(len(top_features))],
                  'keywords_dict':{top_features[j]:weights[j] for j in range(len(top_features))}})
  # format of keywords_weightings?
  i+=1

topic_df = pd.DataFrame(results)

# Export Data
write_data('nmf_topic_data_20mar11-21jun16_('+str(n_components)+','+str(n_top_words)+').json', topic_df, CHECKPOINT_PATH)

# Plot 
plot_top_words(nmf, tfidf_feature_names, n_top_words,
               'Topics in NMF model (Frobenius norm)')

